# Декораторы в Python

**Декораторы** — это обёртки вокруг Python-функций (или классов), которые изменяют работу того, к чему они применяются. Чаще всего они изображаются как "@my_decorator"над декорируемой функцией.

* Декоратор — это всего лишь функция (которая, в качестве аргумента, принимает другую функцию).
* Декораторами пользуются, помещая их имя со знаком @ перед определением функции, а не вызывая их.

В Python декораторы используются, в основном, для декорирования функций (или, соответственно, методов). Возможно, одним из самых распространённых декораторов является декоратор @property:

In [ ]:
class Rectangle:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    @property
    def area(self):
        return self.a * self.b


rect = Rectangle(5, 6)
print(rect.area)

В последней строке кода, мы можем обратиться к члену `area` экземпляра класса Rectangle как к атрибуту. То есть — нам не нужно вызывать метод `area`. Вместо этого при обращении к `area` как к атрибуту (то есть — без использования скобок, ()), соответствующий метод вызывается неявным образом. Это возможно благодаря декоратору `@property`.

Размещение конструкции @property перед определением функции равносильно использованию конструкции вида `area = property(area)`. Другими словами, `property` — это функция, которая принимает другую функцию в качестве аргумента и возвращает ещё одну функцию. Именно этим и занимаются декораторы.

В результате оказывается, что декоратор меняет поведение декорируемой функции.

Предположим, имеется функция, которую мы хотим запустить повторно в том случае, если при её первом запуске произойдёт сбой. То есть — нам нужна функция (декоратор, имя которого, retry, можно перевести как «повтор»), которая вызывает нашу функцию один или два раза (это зависит от того, возникнет ли ошибка при первом вызове функции).

In [2]:
from typing import Callable
import time


def retry(func: Callable):
    def _wrapper(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except:
            time.sleep(1)
            func(*args, **kwargs)

    return _wrapper


@retry
def might_fail():
    print("might_fail")
    raise Exception


might_fail()

might_fail
might_fail


Exception: 

Наш декоратор носит имя `retry`. Он принимает в виде аргумента (`func`) любую функцию. Внутри декоратора определяется новая функция (`_wrapper`), после чего осуществляется возврат этой функции. Тому, кто впервые видит код декоратора, может показаться непривычным объявление одной функции внутри другой функции. Но это — совершенно корректная синтаксическая конструкция, следствием применения которой является тот полезный для нас факт, что функция `_wrapper` видна лишь внутри пространства имён декоратора `retry`.

Напишем ещё один полезный декоратор — timer («таймер»). Он будет измерять время выполнения декорированной с его помощью функции:

In [3]:
import functools
import time


def timer(func):
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        runtime = time.perf_counter() - start
        print(f"{func.__name__} took {runtime:.4f} secs")
        return result

    return _wrapper


@timer
def complex_calculation():
    """Some complex calculation."""
    time.sleep(0.5)
    return 42


print(complex_calculation())

complex_calculation took 0.5006 secs
42


In [19]:
def makebold(fn):
    def wrapped():
        return "<b>" + fn() + "</b>"

    return wrapped


def makeitalic(fn):
    def wrapped():
        return "<i>" + fn() + "</i>"

    return wrapped


@makebold
@makeitalic
def say_words():
    return "Hello!"


say_words()

'<b><i>Hello!</i></b>'

Подытожим:

In [76]:
# Декоратор - это функция, ожидающая ДРУГУЮ функцию в качестве параметра
def my_shiny_new_decorator(a_function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку".
    # Она будет (что бы вы думали?..) обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.

    print("Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать тебе декоратор. ")

    def the_wrapper_around_the_original_function():
        # Поместим здесь код, который мы хотим запускать ДО вызова
        # оригинальной функции
        print("Я - код, который отработает до вызова функции")

        # ВЫЗОВЕМ саму декорируемую функцию
        a_function_to_decorate()

        # А здесь поместим код, который мы хотим запускать ПОСЛЕ вызова
        # оригинальной функции
        print("А я - код, срабатывающий после")

    # На данный момент функция "a_function_to_decorate" НЕ ВЫЗЫВАЛАСЬ НИ РАЗУ

    # Теперь, вернём функцию-обёртку, которая содержит в себе
    # декорируемую функцию, и код, который необходимо выполнить до и после.
    # Всё просто!
    print("Я возвращаю декоратор.")

    return the_wrapper_around_the_original_function


# Представим теперь, что у нас есть функция, которую мы не планируем больше трогать.
def a_stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?..")


a_stand_alone_function()

Я простая одинокая функция, ты ведь не посмеешь меня изменять?..


In [78]:
# Однако, чтобы изменить её поведение, мы можем декорировать её, то есть
# Просто передать декоратору, который обернет исходную функцию в любой код,
# который нам потребуется, и вернёт новую, готовую к использованию функцию:

a_stand_alone_function_decorated = my_shiny_new_decorator(a_stand_alone_function)

Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать тебе декоратор. 
Я возвращаю декоратор.


In [79]:
a_stand_alone_function_decorated()

Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
А я - код, срабатывающий после


Тот же код, только с использованием декораторов:

In [80]:
@my_shiny_new_decorator
def a_stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?..")

Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать тебе декоратор. 
Я возвращаю декоратор.


In [81]:
a_stand_alone_function()

Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
А я - код, срабатывающий после


In [82]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print("Оставь меня в покое")

Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать тебе декоратор. 
Я возвращаю декоратор.


In [83]:
another_stand_alone_function()

Я - код, который отработает до вызова функции
Оставь меня в покое
А я - код, срабатывающий после


Декораторы друг в друга, например так:

In [64]:
def bread(func):
    def wrapper():
        print("</------\>")
        func()
        print("<\______/>")

    return wrapper


def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")

    return wrapper


def sandwich(food="--ветчина--"):
    print(food)


sandwich()
#выведет: --ветчина--
sandwich = bread(ingredients(sandwich))
sandwich()
#выведет:
# </------\>
# #помидоры#
# --ветчина--
# ~салат~
# <\______/>

--ветчина--
</------\>
#помидоры#
--ветчина--
~салат~
<\______/>


In [65]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

</------\>
#помидоры#
--ветчина--
~салат~
<\______/>


Порядок следования декораторов важен:

In [66]:
@ingredients
@bread
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

#помидоры#
</------\>
--ветчина--
<\______/>
~салат~


Передача («проброс») аргументов в декорируемую функцию:


In [67]:
def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):  # аргументы прибывают отсюда
        print("Смотри, что я получил:", arg1, arg2)
        function_to_decorate(arg1, arg2)

    return a_wrapper_accepting_arguments


# Теперь, когда мы вызываем функцию, которую возвращает декоратор,
# мы вызываем её "обёртку", передаём ей аргументы и уже в свою очередь
# она передаёт их декорируемой функции

@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print("Меня зовут", first_name, last_name)


print_full_name("Питер", "Венкман")

Смотри, что я получил: Питер Венкман
Меня зовут Питер Венкман


In [85]:
def decorator_maker_with_arguments(decorator_arg1, decorator_arg2):

    print("Я создаю декораторы! И я получил следующие аргументы:", decorator_arg1, decorator_arg2)

    def my_decorator(func):
        print("Я - декоратор. И ты всё же смог передать мне эти аргументы:", decorator_arg1, decorator_arg2)

        # Не перепутайте аргументы декораторов с аргументами функций!
        def wrapped(function_arg1, function_arg2) :
            print ("Я - обёртка вокруг декорируемой функции.\n"
                  "И я имею доступ ко всем аргументам: \n"
                  "\t- и декоратора: {0} {1}\n"
                  "\t- и функции: {2} {3}\n"
                  "Теперь я могу передать нужные аргументы дальше"
                  .format(decorator_arg1, decorator_arg2,
                          function_arg1, function_arg2))
            return func(function_arg1, function_arg2)

        return wrapped

    return my_decorator

@decorator_maker_with_arguments("Леонард", "Шелдон")
def decorated_function_with_arguments(function_arg1, function_arg2):
    print ("Я - декорируемая функция и я знаю только о своих аргументах: {0}"
           " {1}".format(function_arg1, function_arg2))

decorated_function_with_arguments("Раджеш", "Говард")

Я создаю декораторы! И я получил следующие аргументы: Леонард Шелдон
Я - декоратор. И ты всё же смог передать мне эти аргументы: Леонард Шелдон
Я - обёртка вокруг декорируемой функции.
И я имею доступ ко всем аргументам: 
	- и декоратора: Леонард Шелдон
	- и функции: Раджеш Говард
Теперь я могу передать нужные аргументы дальше
Я - декорируемая функция и я знаю только о своих аргументах: Раджеш Говард


### Декораторы классов

In [68]:
@timer
class MyClass:
    def complex_calculation(self):
        time.sleep(1)
        return 42


my_obj = MyClass()
my_obj.complex_calculation()

MyClass took 0.0000 secs


42

Вспомним о том, что конструкция, начинающаяся с `@` — это всего лишь эквивалент `MyClass = timer(MyClass)`. То есть — декоратор вызывается только когда «вызывают» класс. «Вызов» класса — это создание его экземпляра. Получается, что `timer` вызывается лишь при выполнении строки кода `my_obj = MyClass()`.

При декорировании класса методы этого класса не подвергаются автоматическому декорированию. Проще говоря — использование обычного декоратора для декорирования обычного класса приводит лишь к декорированию конструктора (метод __init__) этого класса.

Но можно поменять поведение всего класса, воспользовавшись другой формой конструктора. Правда, прежде чем об этом говорить, давайте поинтересуемся тем, может ли декоратор работать несколько иначе — то есть можно ли декорировать функцию с помощью класса. Оказывается — это возможно:

In [69]:
class MyDecorator:
    def __init__(self, function):
        self.function = function
        self.counter = 0

    def __call__(self, *args, **kwargs):
        self.function(*args, **kwargs)
        self.counter += 1
        print(f"Called {self.counter} times")


@MyDecorator
def some_function():
    return 42


some_function()
some_function()
some_function()

Called 1 times
Called 2 times
Called 3 times


В ходе работы этого кода происходит следующее:
* Функция `__init__` вызывается при декорировании `some_function`. Тут, снова, не забываем о том, что использование декоратора — это аналог конструкции `some_function = MyDecorator(some_function)`.
* Функция `__call__` вызывается при использовании экземпляра класса, например — при вызове функции. Функция `some_function` — это теперь экземпляр класса `MyDecorator`, но использовать мы её при этом планируем как функцию. За это отвечает магический метод `__call__`, в имени которого используются два символа подчёркивания.

### Декораторы методов

Один из важных фактов, которые следует понимать, заключается в том, что функции и методы в Python'e — это практически одно и то же, за исключением того, что методы всегда ожидают первым параметром ссылку на сам объект (`self`). Это значит, что мы можем создавать декораторы для методов так же, как и для функций, просто не забывая про `self`.

In [70]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie = lie - 3  # действительно, дружелюбно - снизим возраст ещё сильней :-)
        return method_to_decorate(self, lie)

    return wrapper


class Woman(object):

    def __init__(self):
        self.age = 32

    @method_friendly_decorator
    def sayYourAge(self, lie):
        print("Мне {}, а ты бы сколько дал?".format(self.age + lie))


lucy = Woman()
lucy.sayYourAge(-3)

Мне 26, а ты бы сколько дал?


Можно описать такие декораторы, которые могли бы оборачивать любые функции и методы:

In [71]:
def a_decorator_passing_arbitrary_arguments(function_to_decorate):
    # Данная "обёртка" принимает любые аргументы
    def a_wrapper_accepting_arbitrary_arguments(*args, **kwargs):
        print("Передали ли мне что-нибудь?:")
        print(args, kwargs)
        return function_to_decorate(*args, **kwargs)

    return a_wrapper_accepting_arbitrary_arguments

In [72]:
@a_decorator_passing_arbitrary_arguments
def function_with_no_argument():
    print("Python is cool, no argument here.")  # оставлено без перевода, хорошая игра слов:)


function_with_no_argument()

Передали ли мне что-нибудь?:
() {}
Python is cool, no argument here.


In [73]:
@a_decorator_passing_arbitrary_arguments
def function_with_arguments(a, b, c):
    print(a, b, c)


function_with_arguments(1, 2, 3)

Передали ли мне что-нибудь?:
(1, 2, 3) {}
1 2 3


In [74]:
@a_decorator_passing_arbitrary_arguments
def function_with_named_arguments(a, b, c, platypus="Почему нет?"):
    print("Любят ли {}, {} и {} утконосов? %s".format(a, b, c, platypus))

function_with_named_arguments("Билл", "Линус", "Стив", platypus="Определенно!")

Передали ли мне что-нибудь?:
('Билл', 'Линус', 'Стив') {'platypus': 'Определенно!'}
Любят ли Билл, Линус и Стив утконосов? %s


In [75]:
class Woman:

    def __init__(self):
        self.age = 31

    @a_decorator_passing_arbitrary_arguments
    def sayYourAge(self, lie=-3):  # Теперь мы можем указать значение по умолчанию
        print("Мне {}, а ты бы сколько дал?".format(self.age + lie))


lucy = Woman()
lucy.sayYourAge()

Передали ли мне что-нибудь?:
(<__main__.Woman object at 0x000001C51D73BB50>,) {}
Мне 28, а ты бы сколько дал?


#### property

In [5]:
class MyClass:
    def __init__(self, t):
        self._temp = t

    @property
    def temperature(self):
        return self._temp

    @temperature.setter
    def temperature(self, t):
        print(f"Установка температуры в значение {t}")
        self._temp = t


c = MyClass(500)

c.temperature = 1

Установка температуры в значение 1


`@property` — это, скорее всего, один из самых популярных Python-декораторов. Его цель заключается в том, чтобы обеспечить доступ к результатам вызова метода класса в такой форме, как будто этот метод является атрибутом. Конечно, существует и альтернатива `@property`, что позволяет, при выполнении операции присваивания значения, самостоятельно выполнять вызов метода.

In [8]:
class MyClass:
    def __init__(self, x):
        self.x = x

    @property
    def x_doubled(self):
        return self.x * 2

    @x_doubled.setter
    def x_doubled(self, x_doubled):
        self.x = x_doubled // 2


my_object = MyClass(5)
print(my_object.x_doubled)  #  10
print(my_object.x)  #  5
my_object.x_doubled = 100  #
print(my_object.x_doubled)  #  100
print(my_object.x)  #  50

10
5
100
50


#### staticmethod

Ещё один широко известный декоратор — это @staticmethod. Он используется в ситуациях, когда надо вызвать функцию, объявленную в классе, не создавая при этом экземпляр данного класса:

In [14]:
from datetime import date


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)

    @staticmethod
    def is_adult(age):
        return age >= 18


print(Person.is_adult(10))
print(Person.from_birth_year("Ivan", 1994))

False


#### functools.cache

In [12]:
from functools import cache


@cache
def complex_calculations(n):
    print("complex_calculation")
    return 42


print(complex_calculations(1))
print(complex_calculations(1))
print(complex_calculations(1))

complex_calculation
42
42
42


Теперь, при попытке вызова `complex_calculations()`, Python, перед вызовом функции something_complex, проверяет, имеется ли кешированный результат её работы. Если результат её вызова имеется в кеше — `something_complex` не придётся вызывать дважды.

#### dataclass

In [15]:
from dataclasses import dataclass


@dataclass
class InventoryItem:
    name: str
    unit_price: float
    quantity: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity


item = InventoryItem(name="", unit_price=12, quantity=100)
print(item.total_cost())  # 1200

1200


Декоратор `@dataclass` просто снимает с нас нагрузку по написанию конструктора класса, позволяя избежать ручного написания кода, подобного следующему:

In [ ]:
# ...
#     def __init__(self, name, unit_price, quantity):
#         self.name = name
#         self.unit_price = unit_price
#         self.quantity = quantity
# ...

### Выводы

Декораторы могут быть использованы для расширения возможностей функций из сторонних библиотек (код которых мы не можем изменять).

In [125]:
from time import time


def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции.
    """
    def wrapper(*args, **kwargs):
        t = time()
        res = func(*args, **kwargs)
        print(func.__name__, time() - t)
        return res

    return wrapper


def logging(func):
    """
    Декоратор, логирующий работу кода.
    (хорошо, он просто выводит вызовы, но тут могло быть и логирование!)
    """

    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res

    return wrapper


def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов
    декорируемой функции.
    """

    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print("{0} была вызвана: {1}x".format(func.__name__, wrapper.count))
        return res

    wrapper.count = 0

    return wrapper


@benchmark
@logging
@counter
def reverse_string(string):
    return string[::-1]

In [126]:
s = reverse_string("А роза упала на лапу Азора")
print(s)

reverse_string была вызвана: 1x
reverse_string ('А роза упала на лапу Азора',) {}
reverse_string 0.0
арозА упал ан алапу азор А


### Прочее

Чтобы избежать переписывания важной информации, убедитесь в использовании @functools.wraps:

In [136]:
def my_decorator(func):
    def call_func(*args):
        """doc of call_func"""

        return func(*args)

    return call_func


@my_decorator
def f(x):
    """does some math"""
    return x + x * x


print(f.__name__)
print(f.__doc__)

call_func
doc of call_func


Чтобы не было такого поведения, используется декоратор `wraps`:

In [137]:
from functools import wraps


def my_decorator(func):
    @wraps(func)
    def call_func(*args):
        """doc of call_func"""

        return func(*args)

    return call_func


@my_decorator
def f(x):
    """does some math"""
    return x + x * x


print(f.__name__)
print(f.__doc__)

f
does some math
